In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [56]:

data=pd.read_csv('BTC-USD.csv',parse_dates=["Date"],sep=',')

#convert dates from string to datetime objects
data["Date"]=pd.to_datetime(data["Date"])

#keep only the date part of datetime object
data["Date"]=data["Date"].dt.date

#drop the nan values
data.isna().sum()
data.dropna(axis=0,how='any',inplace=True)
data.isna().sum()


#change volume to integer
#data["Volume"]=data["Volume"].astype('int64')


data.shape









(1823, 7)

# Preprocessing

### Testing for multicollinearity
Running the cell we see the close relationship btw Close and Adj Close

Add a dummy variable for time

In [71]:
data['Time_dummy']=np.arange(len(data.index))
data['Time_dummy']=data['Time_dummy'].astype('float64')
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Time_dummy
0,2016-12-13,780.646973,788.460022,777.961975,780.556030,780.556030,81645600.0,0.0
1,2016-12-14,780.005005,782.033997,776.838989,781.481018,781.481018,75979000.0,1.0
2,2016-12-15,780.070007,781.434998,777.802002,778.088013,778.088013,81580096.0,2.0
3,2016-12-16,778.963013,785.031982,778.963013,784.906982,784.906982,83608200.0,3.0
4,2016-12-17,785.166016,792.508972,784.864014,790.828979,790.828979,78989800.0,4.0
5,2016-12-18,791.007996,794.737000,788.026001,790.530029,790.530029,60524400.0,5.0
6,2016-12-19,790.692017,793.611023,790.320007,792.713989,792.713989,74886400.0,6.0
7,2016-12-20,792.247009,801.336975,791.497009,800.875977,800.875977,99629296.0,7.0
8,2016-12-21,800.643982,834.281006,799.405029,834.281006,834.281006,155576000.0,8.0
9,2016-12-22,834.179993,875.781982,834.148987,864.539978,864.539978,200027008.0,9.0


In [72]:
prepro=preprocessing.MinMaxScaler()
data_float=data.loc[:, data.dtypes=='float64']
data_scaled=pd.DataFrame(prepro.fit_transform(data_float), index=data_float.index,columns=data_float.columns)
data_scaled

,Open,High,Low,Close,Adj Close,Volume,Time_dummy
0,0.000082,0.000103,0.000338,0.000042,0.000042,0.000060,0.000000
1,0.000072,0.000009,0.000321,0.000056,0.000056,0.000044,0.000549
2,0.000073,0.000000,0.000336,0.000005,0.000005,0.000060,0.001098
3,0.000057,0.000053,0.000354,0.000107,0.000107,0.000066,0.001647
4,0.000150,0.000163,0.000444,0.000196,0.000196,0.000053,0.002195
...,...,...,...,...,...,...,...
1822,0.743920,0.735437,0.710121,0.702126,0.702126,0.084190,0.997805
1823,0.701869,0.723940,0.705021,0.695706,0.695706,0.088073,0.998353
1824,0.696215,0.715749,0.703782,0.727436,0.727436,0.073282,0.998902
1825,0.727518,0.734374,0.730958,0.738453,0.738453,0.062349,0.999451


##### This relationship is also verified by this:

In [83]:
data_duplicate=data_scaled.T.duplicated()
print(data_duplicate)

#drop tha adj close column but keep the same dataframe
data_scaled.drop(["Adj Close"],axis=1,inplace=True)
print(data_scaled.head(10))

Open          False
High          False
Low           False
Close         False
Adj Close      True
Volume        False
Time_dummy    False
dtype: bool
       Open      High       Low     Close    Volume  Time_dummy
0  0.000082  0.000103  0.000338  0.000042  0.000060    0.000000
1  0.000072  0.000009  0.000321  0.000056  0.000044    0.000549
2  0.000073  0.000000  0.000336  0.000005  0.000060    0.001098
3  0.000057  0.000053  0.000354  0.000107  0.000066    0.001647
4  0.000150  0.000163  0.000444  0.000196  0.000053    0.002195
5  0.000237  0.000196  0.000492  0.000191  0.000000    0.002744
6  0.000232  0.000179  0.000527  0.000224  0.000041    0.003293
7  0.000256  0.000293  0.000545  0.000346  0.000111    0.003842
8  0.000381  0.000777  0.000665  0.000846  0.000271    0.004391
9  0.000884  0.001387  0.001195  0.001299  0.000398    0.004940


#### Split the data in training and test

In [86]:
from sklearn.model_selection import train_test_split
x=data_scaled[['Volume', 'Time_dummy']]
y=data_scaled['Close']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=5849)

from sklearn.linear_model import LinearRegression
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x_train,y_train)

y_hat=LR.predict(x_test)
y_hat

array([ 6.40641400e-02,  1.60809579e-01, -6.86764598e-02,  3.48731628e-01,
       -8.02495245e-02,  1.15467705e-01,  4.42770667e-01,  2.93148362e-01,
        3.53202644e-01,  6.17150400e-02,  9.04410468e-02,  3.46693923e-01,
        6.10973302e-02, -4.35823592e-02,  1.66061134e-01,  2.36321081e-01,
        4.40711823e-02,  7.30272445e-02,  3.83218703e-01,  4.96893859e-01,
        5.93329769e-01,  5.28523352e-01,  4.66437150e-01,  3.86387162e-01,
        6.27764614e-02, -2.06750630e-02,  3.77385296e-01, -8.19808589e-02,
        6.51780886e-02,  3.16163414e-01,  6.87633219e-02,  7.33442627e-02,
        4.68921955e-01,  5.15731878e-01,  4.93123046e-02, -8.11025443e-02,
       -8.14341424e-02,  8.84721089e-03,  3.13208020e-01,  7.66500562e-02,
       -4.92688906e-02,  2.41382306e-01,  4.65837224e-01, -4.60879690e-02,
        3.80686115e-01,  5.97538371e-01,  2.44738375e-01,  3.59512504e-01,
        5.46089250e-01,  3.40324220e-01,  3.42269744e-01, -6.04458114e-02,
        4.72681796e-01,  

In [88]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# predicting the accuracy score
score=r2_score(y_test,y_hat)
print('r2 score is: ', score)
print('mean s error is   ',mean_squared_error(y_test,y_hat))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_hat)))

r2 score is:  0.606945461538819
mean s error is    0.024441248966826944
root_mean_squared error of is== 0.15633697248836229
